### BOM Dataset with Footnotes

In [9]:
import requests
from operator import itemgetter
import pandas as pd
url = 'https://openscriptureapi.org/api/scriptures/v1/lds/en/chapter/1nephi1'
body = {
    "includeExtras": {
        "footnotes" : True
    }
}

response = requests.get(url,json=body)

if(response.status_code == 200):
  chapter = response.json()['chapter']
  bookTitle, number, verses = itemgetter('bookTitle', 'number', 'verses')(chapter)

  print('Got', bookTitle, number)
  for i, verse in enumerate(verses):
    print(i + 1, verse['text'])
else: print(f'Status code: {response.status_code}')



Got 1 Nephi 1
1 I, Nephi, having been born of goodly parents, therefore I was taught somewhat in all the learning of my father; and having seen many afflictions in the course of my days, nevertheless, having been highly favored of the Lord in all my days; yea, having had a great knowledge of the goodness and the mysteries of God, therefore I make a record of my proceedings in my days.
2 Yea, I make a record in the language of my father, which consists of the learning of the Jews and the language of the Egyptians.
3 And I know that the record which I make is true; and I make it with mine own hand; and I make it according to my knowledge.
4 For it came to pass in the commencement of the first year of the reign of Zedekiah, king of Judah, (my father, Lehi, having dwelt at Jerusalem in all his days); and in that same year there came many prophets, prophesying unto the people that they must repent, or the great city Jerusalem must be destroyed.
5 Wherefore it came to pass that my father, Le

In [10]:
data = response.json()
#print("Text:", data['chapter']['verses'][0]['text'])
print("\nFootnotes:", data['chapter']['verses'][0]['footnotes'][2]['footnote'])



Footnotes: Mosiah 1:2 (2–3); D&C 68:25 (25, 28). TG Honoring Father and Mother.


In [11]:
data['volume']['title']

'Book of Mormon'

In [13]:
def call_api(url, df:pd.DataFrame=None):
    if df is None:
        columns = ['chapterTitle', 'chapterNum', 'verseNum', 'text', 'footnotes']
        df = pd.DataFrame(columns=columns)

    url_template = 'https://openscriptureapi.org/api/scriptures/v1/lds/en/chapter/'
    body = {
        "includeExtras": {
            "footnotes" : True
        }
    }

    response = requests.get(url,json=body)

    if(response.status_code == 200):
        ### add logic to add it to a df here ###

        data = response.json()
        if data['volume']['title'] != 'Book of Mormon':
            print(f"End of the BoM. Hit {data['volume']['title']}")
            return df

        chapter = data['chapter']['bookTitle'] + ' ' + str(data['chapter']['number'])
        print(f'chapter: {chapter}')
        verses = data['chapter']['verses']
        verseNum = 1
        for verse in verses:
            footnotes_list = []
            for footnote in verse['footnotes']:
                if ';' in footnote['footnote']:
                    parts = footnote['footnote'].split(';')
                    footnotes_list.extend(parts)
                else:
                    footnotes_list.append(footnote['footnote'])

            new_row = {
                'chapterTitle': chapter,
                'chapterNum' : data['chapter']['number'],
                'verseNum' : verseNum,
                'text': verse['text'],
                'footnotes': footnotes_list
            }
            df.loc[len(df)] = new_row
            verseNum += 1
        #print("Success with chapter", data['chapter']['bookTitle'], data['chapter']['number'])
        return call_api(url_template + data['nextChapterId'],df)
    else: (print("Status Code: ", response.status_code))

df = call_api(url)
df.head()

chapter: 1 Nephi 1
chapter: 1 Nephi 2
chapter: 1 Nephi 3
chapter: 1 Nephi 4
chapter: 1 Nephi 5
chapter: 1 Nephi 6
chapter: 1 Nephi 7
chapter: 1 Nephi 8
chapter: 1 Nephi 9
chapter: 1 Nephi 10
chapter: 1 Nephi 11
chapter: 1 Nephi 12
chapter: 1 Nephi 13
chapter: 1 Nephi 14
chapter: 1 Nephi 15
chapter: 1 Nephi 16
chapter: 1 Nephi 17
chapter: 1 Nephi 18
chapter: 1 Nephi 19
chapter: 1 Nephi 20
chapter: 1 Nephi 21
chapter: 1 Nephi 22
chapter: 2 Nephi 1
chapter: 2 Nephi 2
chapter: 2 Nephi 3
chapter: 2 Nephi 4
chapter: 2 Nephi 5
chapter: 2 Nephi 6
chapter: 2 Nephi 7
chapter: 2 Nephi 8
chapter: 2 Nephi 9
chapter: 2 Nephi 10
chapter: 2 Nephi 11
chapter: 2 Nephi 12
chapter: 2 Nephi 13
chapter: 2 Nephi 14
chapter: 2 Nephi 15
chapter: 2 Nephi 16
chapter: 2 Nephi 17
chapter: 2 Nephi 18
chapter: 2 Nephi 19
chapter: 2 Nephi 20
chapter: 2 Nephi 21
chapter: 2 Nephi 22
chapter: 2 Nephi 23
chapter: 2 Nephi 24
chapter: 2 Nephi 25
chapter: 2 Nephi 26
chapter: 2 Nephi 27
chapter: 2 Nephi 28
chapter: 2 Nephi 2

,chapterTitle,chapterNum,verseNum,text,footnotes
0,1 Nephi 1,1,1,"I, Nephi, having been born of goodly parents, ...","[TG Birthright., Prov. 22:1., Mosiah 1:2 (2–3)..."
1,1 Nephi 1,1,2,"Yea, I make a record in the language of my fat...","[Mosiah 1:4, Morm. 9:32 (32–33).]"
2,1 Nephi 1,1,3,And I know that the record which I make is tru...,"[1 Ne. 14:30, 2 Ne. 25:20, Mosiah 1:6, Alma..."
3,1 Nephi 1,1,4,For it came to pass in the commencement of the...,"[1 Ne. 2:4, Mosiah 6:4., 2 Kgs. 24:18, 2 Chr..."
4,1 Nephi 1,1,5,"Wherefore it came to pass that my father, Lehi...","[Jer. 29:13, James 5:16, 2 Ne. 4:24 (23–25).]"


In [14]:
df.to_csv('bom_dataset_with_footnotes.csv')